# CP321 Project (20 Marks)

Student #1:

Name:        Malika Sharma

Student ID:  210 - 37


Student #2:

Name:        Ruveyda Nur Kizmaz

Student ID:  210960430

### Introduction & Motivation

Introduction: xxx

Our Motivation:  xxxxxxx  20% of our mark is enough motivation to do this work

### Propose Approach

Our Question: What are the trends in student achievements for reading, writing, and mathematics in grades 3, 6, and 9 EQAO assessments across Ontario from 2021 to 2023?

Related Dataset:
https://data.ontario.ca/datasetschool-information-and-student-demographics 

Dataset Description:

Data Processing:     ??


In [5]:
# Our code starts from here

# All imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# test opening file
# Required download: to be able to open xlsx files:
# pip install openpyxl
df = pd.read_excel(r'data/new_sif_data_table_2021_22prelim_en_march2024.xlsx')
print(df)



     Board Number       Board Name              Board Type School Number  \
0          B28010       Algoma DSB  Pub Dist Sch Brd (E/F)        902344   
1          B28010       Algoma DSB  Pub Dist Sch Brd (E/F)        019186   
2          B28010       Algoma DSB  Pub Dist Sch Brd (E/F)        043362   
3          B28010       Algoma DSB  Pub Dist Sch Brd (E/F)        054542   
4          B28010       Algoma DSB  Pub Dist Sch Brd (E/F)        490822   
...           ...              ...                     ...           ...   
4911       B66095  York Region DSB  Pub Dist Sch Brd (E/F)        549380   
4912       B66095  York Region DSB  Pub Dist Sch Brd (E/F)        954292   
4913       B66095  York Region DSB  Pub Dist Sch Brd (E/F)        617318   
4914       B66095  York Region DSB  Pub Dist Sch Brd (E/F)        618896   
4915       B66095  York Region DSB  Pub Dist Sch Brd (E/F)        624101   

                                       School Name School Type  \
0     Algoma Educatio

In [ ]:
# MALIKA
""" 
Use grouped bar charts to display the average scores 
for each subject (reading, writing, and mathematics) 
side by side for each year (2021, 2022, and 2023).
"""

In [ ]:
# NUR
""" 
A line chart for each subject showing the trend 
over the three years for each grade.
"""





In [ ]:
# NUR
"""
Stacked bar charts could be used to show 
the proportion of students achieving 
each level of proficiency 
(e.g., not meeting, meeting, exceeding expectations) 
in each subject for each grade over the three years.
"""

In [ ]:
# NUR
"""
A heatmap could represent the concentration 
of scores across different grades and subjects.
"""

In [ ]:
# MALIKA
"""
Boxplots would allow for a comparison of the distribution 
of scores for each subject across the three grades, 
highlighting the median, quartiles, and 
any potential outliers in the data.
"""